# Mimim iii to Ecmo Card Mapping
## Setup
### Install  libs 
Only run pip install one when first installing notebook.

In [12]:
#!pip install pyathena
#! pip install PyAthena[SQLAlchemy]

### Imports  and settings

In [13]:
from io import StringIO
import contextlib
import yaml
from urllib.parse import quote_plus
import numpy as np
import pandas as pd
import boto3
from pyathena import connect
from sqlalchemy import create_engine
from sqlalchemy.types import String

import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

S3_BUCKET = 'ecmo-athena-query-grr' # bucket for all created tabel data
REGION='ap-southeast-2' # AWS region for all data
WRITE_SCHEMA = "echmocard" # write schema is the schema where all intermediary and final mapped tabels are created
READ_SCHEMA = "mimic3-sampleset" # read schema is where mimic3 raw data is

## Athena
We will use Athena to do all of the mapping work by creating tables of mapped mimic 3 data.

### Athena connection
First lets get the Athena connection to the mimic 3 schema (the read schema)

In [14]:
def athena_connection(schema):
    return connect(s3_staging_dir=f's3://{S3_BUCKET}/athena/stage/', region_name=REGION, schema_name=schema)

connection = athena_connection(READ_SCHEMA)

### Create table as select
For large table inserts, we dont want to load data into dataframes locally and then write them back into Athena, so we we need a function that will overwrite tables in Athena dirctly from a select. This is a 3 step process. First drop the existing table from the Athena catalog, remove its data from S3, and then re-create the table from the select statement.

In [15]:
def athena_ctas(table, query):    
    path = f'athena/table/{WRITE_SCHEMA}/{table}'
    connection = athena_connection(WRITE_SCHEMA)

    # drop the table from catalog
    sql = f'DROP TABLE {table}'
    print(sql)
    connection.cursor().execute(sql)
    
    # remove s3 data if it exists
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(S3_BUCKET)
    for obj in bucket.objects.filter(Prefix=path):
        print(f'DELETE s3://{S3_BUCKET}/{obj.key}')
        s3.Object(bucket.name, obj.key).delete()
        
    # create table as select
    sql = f"CREATE TABLE {table}\nWITH (external_location='s3://{S3_BUCKET}/{path}/') AS {query}"
    print(sql)
    connection.cursor().execute(sql)
    
def sample(connection, table, limit=5):
    return pd.read_sql(f'select * from "{WRITE_SCHEMA}".{table} limit {limit}', connection)

## ICD9 Mapping
For many of the queries, we will need to have ICD9 codes mapped to ecmo attributes. This basic mnapping is done by reading a configuration file that lists the ecmo field name, all the ICD9 code that map to it.

### Read icd9 mapping file
Read the mappings yaml file [icd9 mappings file](./icd9_mappings.yaml)

In [16]:
with open('icd9_mappings.yaml', 'r') as stream:    
    icd9_mappings = yaml.safe_load(stream)

### Expand mapping codes
ICD9 codes are of the format <3 digit parent code><1 digit subcode><1 digit sub-subcode>. As opposed to specifying all the codes that map to an ecmocard field in the yaml config, specify the parent code and the logic below will expand the list of mappings to all possible sub codes.

In [17]:
expanded_codes = []
mapping = []
for key,mappings in icd9_mappings.items():    
    for value in mappings:
        value = str(value)
        split = value.find(' ')
        code = value[0:split]
        description = value[split+1:]
        codes = [code]
        while len(codes[-1]) < 5:
            expand = []
            for code in codes:
                expand = expand + [code + str(x) for x in range(0, 9)] 
            codes = codes + expand        
        mapping = mapping + [key]*len(codes)
        expanded_codes = expanded_codes + codes
        
mappings = pd.DataFrame({'icd9_code': expanded_codes, 'mapping': mapping})
mappings = mappings.set_index('icd9_code')#.drop_duplicates()

icd9_diagnosis = pd.read_sql('select * from d_icd_diagnoses', connection)
icd9_diagnosis = icd9_diagnosis.set_index('icd9_code')
encode = lambda x:  x.replace("'", "") .replace(',', ' ') # strings causing issues on insert
icd9_diagnosis['short_title'] = icd9_diagnosis['short_title'].apply(encode)
icd9_diagnosis['long_title'] = icd9_diagnosis['long_title'].apply(encode)
mapped = icd9_diagnosis.join(mappings)
mapped = mapped[mapped['mapping'].notna()]
mapped.tail(10)

,row_id,short_title,long_title,mapping
icd9_code,,,,
V275,13673,Del-mult birth all live,Outcome of delivery other multiple birth all...,postpart_rptestcd
V275,13673,Del-mult birth all live,Outcome of delivery other multiple birth all...,postpart_rptestcd
V275,13673,Del-mult birth all live,Outcome of delivery other multiple birth all...,pregout_rptestcd_live
V276,13674,Del-mult brth some live,Outcome of delivery other multiple birth som...,postpart_rptestcd
V276,13674,Del-mult brth some live,Outcome of delivery other multiple birth som...,postpart_rptestcd
V276,13674,Del-mult brth some live,Outcome of delivery other multiple birth som...,pregout_rptestcd_still
V277,13675,Del-mult birth all sb,Outcome of delivery other multiple birth all...,postpart_rptestcd
V277,13675,Del-mult birth all sb,Outcome of delivery other multiple birth all...,postpart_rptestcd
V277,13675,Del-mult birth all sb,Outcome of delivery other multiple birth all...,pregout_rptestcd_still


### Create ICD9 Mapping Table
After expanding the mappings, create an icd9 to ecmocard mapping table in Athena. This will create parquet files in S3 and update the glue data catalog.

In [18]:
write_engine = create_engine(
        f'awsathena+rest://:@athena.{REGION}.amazonaws.com:443/{WRITE_SCHEMA}?' +
        f's3_staging_dir=s3://{S3_BUCKET}/athena/table/&' +
        f's3_dir=s3://{S3_BUCKET}/athena/table/&compression=snappy'
)

df = mapped.drop(columns=['short_title', 'long_title', 'row_id']).reset_index()
df.to_sql(
        'icd9_map',
        write_engine,  
        index=False, 
        if_exists='replace', 
        method='multi',
        schema=WRITE_SCHEMA,
        dtype={
            "mapping": String(30),
            "icd9_code": String(10)
        }
    ) 

pd.read_sql('select * from echmocard.icd9_map limit 10', connection)

,icd9_code,mapping
0,0340,sorethroat_ceoccur_v2
1,0340,sorethroat_ceoccur_v2
2,0341,sorethroat_ceoccur_v2
3,0341,sorethroat_ceoccur_v2
4,042,aidshiv_mhyn
5,1400,malignantneo_mhyn
6,1400,malignantneo_mhyn
7,1401,malignantneo_mhyn
8,1401,malignantneo_mhyn
9,1403,malignantneo_mhyn


## Create intermediary tables
This section will perfrom initial mappings form mimic3 tables to intermediarry tables that can be later combined to produce the various ecmo card sections.

### Diagnosis table
Lets us the above function to join the mimic iii diagnosis table for all subject to the icd9 mapping table to get the diagnosis mapped. We will generate the query uign python and the mapping columns names.

In [19]:
mapped_columns = mapped['mapping'].unique()
bool_sql = ',\n  '.join([f"bool_or(mapping = '{m}') AS {m}" for m in mapped_columns])

athena_ctas('diagnoses', f'''
    SELECT 
        hadm_id,
        subject_id,
        {bool_sql},
        array_except(zip_with(array_agg(mimic_diagnoses.icd9_code), array_agg(mapping), (c,m) -> if(m is NULL, c)), ARRAY[null]) as unmapped,
        array_agg(mimic_diagnoses.icd9_code) AS all_codes,
        array_agg(mapping) AS all_mappings
    FROM "{READ_SCHEMA}".diagnoses_icd AS mimic_diagnoses
    LEFT JOIN icd9_map USING (icd9_code)
    GROUP BY (hadm_id, subject_id)
    ORDER BY subject_id
''')

DROP TABLE diagnoses
DELETE s3://ecmo-athena-query-grr/athena/table/echmocard/diagnoses/20200506_145245_00005_hr7kx_5d59b545-a04a-4cbf-8773-0dc9563b1d11
DELETE s3://ecmo-athena-query-grr/athena/table/echmocard/diagnoses/20200506_145245_00005_hr7kx_6f5b3201-6623-428e-97f1-2ba84bafbee6
CREATE TABLE diagnoses
WITH (external_location='s3://ecmo-athena-query-grr/athena/table/echmocard/diagnoses/') AS 
    SELECT 
        hadm_id,
        subject_id,
        bool_or(mapping = 'sorethroat_ceoccur_v2') AS sorethroat_ceoccur_v2,
  bool_or(mapping = 'aidshiv_mhyn') AS aidshiv_mhyn,
  bool_or(mapping = 'malignantneo_mhyn') AS malignantneo_mhyn,
  bool_or(mapping = 'diabetes_mhyn') AS diabetes_mhyn,
  bool_or(mapping = 'diabetiscomp_mhyn') AS diabetiscomp_mhyn,
  bool_or(mapping = 'malnutrition_mhyn') AS malnutrition_mhyn,
  bool_or(mapping = 'obesity_mhyn') AS obesity_mhyn,
  bool_or(mapping = 'chronhaemo_mhyn') AS chronhaemo_mhyn,
  bool_or(mapping = 'smoker') AS smoker,
  bool_or(mapping = 'chr

In [20]:
left = []
visible_columns = (
    'aidshiv_mhyn,malignantneo_mhyn,diabetes_mhyn,diabetiscomp_mhyn,'+
    'malnutrition_mhyn,obesity_mhyn,chronhaemo_mhyn,chronicneu_mhyn,'+
    'dementia_mhyn,asthma_mhyn,modliver_mhyn,renal_mhyn'
).split(',')
for item in mapped_columns:
    if item not in visible_columns:
        left.append(item)
print(','.join(left))

sorethroat_ceoccur_v2,smoker,seizures_cecoccur_v2,conjunct_ceoccur_v2,earpain_ceoccur_v2,rheumatology_mhyr,chroniccard_mhyn,bleed_ceoccur_v2,runnynose_ceoccur_v2,chronicpul_mhyn,skinulcers_ceoccur_v2,jointpain_ceoccur_v2,myalgia_ceoccur_v2,confusion_ceoccur_v2,fever_ceoccur_v2,fatigue_ceoccur_v2,rash_ceoccur_v2,headache_ceoccur_v2,lymp_ceoccur_v2,shortbreath_ceoccur_v2,wheeze_ceoccur_v2,cough_ceoccur_v2,coughhb_ceoccur_v2,coughsput_ceoccur_v2,chestpain_ceoccur_v2,vomit_ceoccur_v2,diarrhoea_ceoccur_v2,abdopain_ceoccur_v2,former_smoker,pregyn_rptestcd,postpart_rptestcd,pregout_rptestcd_live,pregout_rptestcd_still,oxy_vsyn


In [21]:
sample(connection, 'diagnoses')

,hadm_id,subject_id,sorethroat_ceoccur_v2,aidshiv_mhyn,malignantneo_mhyn,diabetes_mhyn,diabetiscomp_mhyn,malnutrition_mhyn,obesity_mhyn,chronhaemo_mhyn,...,abdopain_ceoccur_v2,former_smoker,pregyn_rptestcd,postpart_rptestcd,pregout_rptestcd_live,pregout_rptestcd_still,oxy_vsyn,unmapped,all_codes,all_mappings
0,126949,10045,False,True,False,True,False,False,False,True,...,False,True,False,False,False,False,False,"[V1042, 44422, 2859, 2724, 5849, V446, 1179, 5...","[V1042, 44422, 2859, 2724, 5849, V446, 1179, 5...","[null, null, null, null, null, null, null, nul..."
1,133110,10046,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,"[V1042, 78039]","[V1042, 78039, 1983, 1983, 1983, 1983, 1983, 1...","[null, null, malignantneo_mhyn, malignantneo_m..."
2,100375,10056,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,"[0389, 99592, 2761, 78552, 5845, 5990, 2765]","[0389, 99592, 2761, 78552, 5845, 5990, 42831, ...","[null, null, null, null, null, null, aidshiv_m..."
3,122098,10059,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,"[07070, 5849, 452, 0389, 99592, 5859, 78552, 7...","[07070, 5723, 5723, 5723, 5723, 5723, 5723, 57...","[null, modliver_mhyn, modliver_mhyn, modliver_..."
4,142582,10059,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,"[07070, 5849, 4019, 7895, 45620]","[07070, 5723, 5723, 5723, 5723, 5723, 5723, 57...","[null, modliver_mhyn, modliver_mhyn, modliver_..."


## ECMO Mapping
Using the mapped tables, we can now map the ecmo card fields for each section of the ecmo card.

### Demographics
#### Logic
* Join the  columns from patients and admissions to get demographics, and diagnosis for pregnency flags
* Ethnicity is mapped from admission table
* Age is calculated by subtracting todays date from DOB. Any age over 90 is set to 90, as the data is random for higher ages
* Pregnancy is determined by ICD9 diagnosis mappings to pregyn_rptestcd
* Postpartum is determined by ICD9 diagnosis mappings to postpart_rptestcd
* If any ICD9 code for birth or multiple birth without any stillbirth, pregout_rptestcd is set to live birth = 1
* Otherwise, If any ICD9 code for stillborn or multiple birth with 1 or more stillborn pregout_rptestcd is set to stillborn = 2
* Otherwise, set to null, for no birth

In [22]:
athena_ctas('demographics', f'''
WITH 
admissions AS (
    SELECT
        subject_id,
        hadm_id,
        ethnicity,
        CASE
            WHEN ethnicity='BLACK/AFRICAN AMERICAN' THEN 2
            WHEN ethnicity='ASIAN' THEN 3
            WHEN ethnicity='HISPANIC OR LATINO' THEN 6
            WHEN ethnicity='HISPANIC/LATINO - PUERTO RICAN' THEN 6
            WHEN ethnicity='WHITE' THEN 7
            WHEN ethnicity='AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE' THEN 8
            WHEN ethnicity='UNKNOWN/NOT SPECIFIED' THEN 10
            ELSE 9 
        END AS ethnic
    FROM "{READ_SCHEMA}".admissions
),

patients AS (
    SELECT 
        subject_id,
        CASE
            WHEN gender = 'F' THEN '2'
            WHEN gender = 'M' THEN '1'
            ELSE '3'
        END AS sexfrom, 
        date_diff('year', date_parse(dob, '%Y-%m-%d %H:%i:%s'), current_date) AS age
    FROM "{READ_SCHEMA}".patients
)

SELECT 
    admissions.subject_id,
    admissions.hadm_id,
    ethnic,
    if (ethnic = 9, ethnicity, NULL) AS other_ethnic,
    
    IF(age<90, age, 90) AS age_estimateyears,
    FLOOR(IF(age<90, age, 90) / 10) AS age_estimate10,
    2 AS age_estimateyearsu,
    sexfrom,
    
    if(pregyn_rptestcd is null, 0, 1) AS pregyn_rptestcd,
    if(postpart_rptestcd is null, 0, 1) AS postpart_rptestcd,
    if(pregout_rptestcd_live is not null, 1, if (pregout_rptestcd_still is not null, 2, null)) AS pregout_rptestcd
FROM admissions
JOIN patients ON (admissions.subject_id = patients.subject_id)
LEFT JOIN diagnoses ON (admissions.hadm_id = diagnoses.hadm_id)
ORDER BY subject_id
''')

DROP TABLE demographics
DELETE s3://ecmo-athena-query-grr/athena/table/echmocard/demographics/20200506_130526_00015_4gbqq_53c2f6b4-39c2-4dc7-b932-ba9021c26164
CREATE TABLE demographics
WITH (external_location='s3://ecmo-athena-query-grr/athena/table/echmocard/demographics/') AS 
WITH 
admissions AS (
    SELECT
        subject_id,
        hadm_id,
        ethnicity,
        CASE
            WHEN ethnicity='BLACK/AFRICAN AMERICAN' THEN 2
            WHEN ethnicity='ASIAN' THEN 3
            WHEN ethnicity='HISPANIC OR LATINO' THEN 6
            WHEN ethnicity='HISPANIC/LATINO - PUERTO RICAN' THEN 6
            WHEN ethnicity='WHITE' THEN 7
            WHEN ethnicity='AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE' THEN 8
            WHEN ethnicity='UNKNOWN/NOT SPECIFIED' THEN 10
            ELSE 9 
        END AS ethnic
    FROM "mimic3-sampleset".admissions
),

patients AS (
    SELECT 
        subject_id,
        CASE
            WHEN gender = 'F' THEN '2'
            WHEN g

In [23]:
sample(connection, 'demographics')

,subject_id,hadm_id,ethnic,other_ethnic,age_estimateyears,age_estimate10,age_estimateyearsu,sexfrom,pregyn_rptestcd,postpart_rptestcd,pregout_rptestcd
0,10006,142345,2,None,-73,-7,2,2,1,1,1
1,10011,105331,10,None,-70,-7,2,2,1,1,1
2,10013,165520,10,None,-18,-1,2,2,1,1,1
3,10017,199207,7,None,-55,-5,2,2,1,1,1
4,10019,177759,7,None,-94,-9,2,1,1,1,1


### Comorbidities 
#### Logic 
* Each column is taken from diagnosis mapping table. If there is an ICD9 Code present during their admission, than the features is flagged as True
* Any ICD9 codes that are in listed in the diagnosis for the admission, but are not mapped are assigned to otherrisktext.

In [24]:
visible_columns = (
    'aidshiv_mhyn,malignantneo_mhyn,diabetes_mhyn,diabetiscomp_mhyn,'+
    'malnutrition_mhyn,obesity_mhyn,chronhaemo_mhyn,chronicneu_mhyn,'+
    'dementia_mhyn,asthma_mhyn,modliver_mhyn,renal_mhyn'
).split(',')
category_sql = ',\n  '.join([f"if({m} is null, 3, if({m}, 1, 2)) AS {m}" for m in visible_columns])
athena_ctas('comorbidities', f'''
SELECT 
  subject_id, 
  hadm_id,
  {category_sql}, 
  if(cardinality(unmapped) > 0, 1, 0) AS other_mhyn,
  CASE
    WHEN smoker THEN 1
    WHEN former_smoker THEN 3
    ELSE 2
  END AS smoking_mhyn,
  unmapped AS otherrisktext      
FROM diagnoses
ORDER BY subject_id
''')

DROP TABLE comorbidities
DELETE s3://ecmo-athena-query-grr/athena/table/echmocard/comorbidities/20200506_144529_00001_dcuby_6eacfa30-d41d-4b6a-84a8-4b535060465d
CREATE TABLE comorbidities
WITH (external_location='s3://ecmo-athena-query-grr/athena/table/echmocard/comorbidities/') AS 
SELECT 
  subject_id, 
  hadm_id,
  if(aidshiv_mhyn is null, 3, if(aidshiv_mhyn, 1, 2)) AS aidshiv_mhyn,
  if(malignantneo_mhyn is null, 3, if(malignantneo_mhyn, 1, 2)) AS malignantneo_mhyn,
  if(diabetes_mhyn is null, 3, if(diabetes_mhyn, 1, 2)) AS diabetes_mhyn,
  if(diabetiscomp_mhyn is null, 3, if(diabetiscomp_mhyn, 1, 2)) AS diabetiscomp_mhyn,
  if(malnutrition_mhyn is null, 3, if(malnutrition_mhyn, 1, 2)) AS malnutrition_mhyn,
  if(obesity_mhyn is null, 3, if(obesity_mhyn, 1, 2)) AS obesity_mhyn,
  if(chronhaemo_mhyn is null, 3, if(chronhaemo_mhyn, 1, 2)) AS chronhaemo_mhyn,
  if(chronicneu_mhyn is null, 3, if(chronicneu_mhyn, 1, 2)) AS chronicneu_mhyn,
  if(dementia_mhyn is null, 3, if(dementia_mhyn,

In [25]:
sample(connection, 'comorbidities')

,subject_id,hadm_id,aidshiv_mhyn,malignantneo_mhyn,diabetes_mhyn,diabetiscomp_mhyn,malnutrition_mhyn,obesity_mhyn,chronhaemo_mhyn,chronicneu_mhyn,dementia_mhyn,asthma_mhyn,modliver_mhyn,renal_mhyn,other_mhyn,smoking_mhyn,otherrisktext
0,10006,142345,1,2,1,2,2,2,2,2,2,2,2,2,1,1,"[40391, E8791, V5867, 2874, E9342, V090, 28529..."
1,10011,105331,2,2,2,2,2,2,1,2,2,2,2,2,1,2,"[30401, 2760, 07030, 570, 07054]"
2,10013,165520,1,1,2,2,2,2,2,2,2,2,2,2,1,2,"[2724, 78551, 486, 41071, 0389, 4582]"
3,10017,199207,2,2,1,2,2,2,1,2,2,2,2,2,1,2,"[8024, 2724, E8788, 4019, 486, 99812, 41511, 8..."
4,10019,177759,2,2,2,2,2,2,2,2,2,2,1,2,1,2,"[30390, 49390, 7299, 4019, 5770, 0389, 99592, ..."


### Onset and admissions
#### Logic
* Hospital admission date and time used (not ICU admission time)
* Admission location indicated other hospital transfer

In [26]:
athena_ctas('onset_and_admissions', f'''
SELECT
    subject_id,
    hadm_id,
    date_format(date_parse(admittime, '%Y-%m-%d %H:%i:%s'), '%Y-%m-%d') AS hostdat,
    date_format(date_parse(admittime, '%Y-%m-%d %H:%i:%s'), '%H:%i:%s') AS hosttim,
    CASE 
        WHEN admission_location = 'TRANSFER FROM OTHER HEALT' THEN 2 
        WHEN admission_location = '** INFO NOT AVAILABLE **' THEN 4
        ELSE 3
    END AS hooccur
FROM
    "{READ_SCHEMA}".admissions
ORDER BY subject_id
''')

DROP TABLE onset_and_admissions
DELETE s3://ecmo-athena-query-grr/athena/table/echmocard/onset_and_admissions/20200506_131910_00003_twpgp_49e3abaa-089e-4959-b7ec-d9a1943fb173
CREATE TABLE onset_and_admissions
WITH (external_location='s3://ecmo-athena-query-grr/athena/table/echmocard/onset_and_admissions/') AS 
SELECT
    subject_id,
    hadm_id,
    date_format(date_parse(admittime, '%Y-%m-%d %H:%i:%s'), '%Y-%m-%d') AS hostdat,
    date_format(date_parse(admittime, '%Y-%m-%d %H:%i:%s'), '%H:%i:%s') AS hosttim,
    CASE 
        WHEN admission_location = 'TRANSFER FROM OTHER HEALT' THEN 2 
        WHEN admission_location = '** INFO NOT AVAILABLE **' THEN 4
        ELSE 3
    END AS hooccur
FROM
    "mimic3-sampleset".admissions
ORDER BY subject_id



In [27]:
sample(connection, 'onset_and_admissions')

,subject_id,hadm_id,hostdat,hosttim,hooccur
0,10006,142345,2164-10-23,21:09:00,3
1,10011,105331,2126-08-14,22:32:00,3
2,10013,165520,2125-10-04,23:36:00,3
3,10017,199207,2149-05-26,17:19:00,3
4,10019,177759,2163-05-14,20:43:00,3


### Admission signs and symptoms
#### Logic
* Each column is taken from diagnosis mapping table. If there is an ICD9 Code present during their admission, than the features is flagged as True

In [28]:
visible_columns = (
    'sorethroat_ceoccur_v2,smoker,seizures_cecoccur_v2,conjunct_ceoccur_v2,earpain_ceoccur_v2,'+
    'rheumatology_mhyr,chroniccard_mhyn,bleed_ceoccur_v2,runnynose_ceoccur_v2,chronicpul_mhyn,'+
    'skinulcers_ceoccur_v2,jointpain_ceoccur_v2,myalgia_ceoccur_v2,confusion_ceoccur_v2,'+
    'fever_ceoccur_v2,fatigue_ceoccur_v2,rash_ceoccur_v2,headache_ceoccur_v2,lymp_ceoccur_v2,'+
    'shortbreath_ceoccur_v2,wheeze_ceoccur_v2,cough_ceoccur_v2,coughhb_ceoccur_v2,coughsput_ceoccur_v2,'+
    'chestpain_ceoccur_v2,vomit_ceoccur_v2,diarrhoea_ceoccur_v2,abdopain_ceoccur_v2,oxy_vsyn'
).split(',')
category_sql = ',\n  '.join([f"if({m} is null, 3, if({m}, 1, 2)) AS {m}" for m in visible_columns])
athena_ctas('admission_signs_and_symptoms', f'''
SELECT 
  subject_id, 
  hadm_id,
  {category_sql}   
FROM diagnoses
ORDER BY subject_id
''')

DROP TABLE admission_signs_and_symptoms
CREATE TABLE admission_signs_and_symptoms
WITH (external_location='s3://ecmo-athena-query-grr/athena/table/echmocard/admission_signs_and_symptoms/') AS 
SELECT 
  subject_id, 
  hadm_id,
  if(sorethroat_ceoccur_v2 is null, 3, if(sorethroat_ceoccur_v2, 1, 2)) AS sorethroat_ceoccur_v2,
  if(smoker is null, 3, if(smoker, 1, 2)) AS smoker,
  if(seizures_cecoccur_v2 is null, 3, if(seizures_cecoccur_v2, 1, 2)) AS seizures_cecoccur_v2,
  if(conjunct_ceoccur_v2 is null, 3, if(conjunct_ceoccur_v2, 1, 2)) AS conjunct_ceoccur_v2,
  if(earpain_ceoccur_v2 is null, 3, if(earpain_ceoccur_v2, 1, 2)) AS earpain_ceoccur_v2,
  if(rheumatology_mhyr is null, 3, if(rheumatology_mhyr, 1, 2)) AS rheumatology_mhyr,
  if(chroniccard_mhyn is null, 3, if(chroniccard_mhyn, 1, 2)) AS chroniccard_mhyn,
  if(bleed_ceoccur_v2 is null, 3, if(bleed_ceoccur_v2, 1, 2)) AS bleed_ceoccur_v2,
  if(runnynose_ceoccur_v2 is null, 3, if(runnynose_ceoccur_v2, 1, 2)) AS runnynose_ceoccur_v2,

In [29]:
sample(connection, 'admission_signs_and_symptoms')

,subject_id,hadm_id,sorethroat_ceoccur_v2,smoker,seizures_cecoccur_v2,conjunct_ceoccur_v2,earpain_ceoccur_v2,rheumatology_mhyr,chroniccard_mhyn,bleed_ceoccur_v2,...,shortbreath_ceoccur_v2,wheeze_ceoccur_v2,cough_ceoccur_v2,coughhb_ceoccur_v2,coughsput_ceoccur_v2,chestpain_ceoccur_v2,vomit_ceoccur_v2,diarrhoea_ceoccur_v2,abdopain_ceoccur_v2,oxy_vsyn
0,10006,142345,2,1,2,2,2,2,1,2,...,2,2,2,2,2,2,2,2,2,2
1,10011,105331,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
2,10013,165520,2,2,2,2,2,2,1,2,...,2,2,2,2,2,2,2,2,2,2
3,10017,199207,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
4,10019,177759,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


### Outcomes 
Now we will create an outcomes table that we can use as labels in a machine learning model.

#### Logic
* If the admissions hospital_expire_flag is set, then the outcome is death = 4. 
* Otherwise, If there is a discharge time, discharged alive = 1 
* Otherwise they are assumed to be hospitalized  = 2

In [ ]:
athena_ctas('outcomes', f'''
SELECT 
    subject_id,
    hadm_id,
    CASE
        WHEN hospital_expire_flag = '1' THEN 4
        WHEN dischtime <> null THEN 1
        ELSE 2
    END AS dsterm
FROM admissions
ORDER BY subject_id
''')

In [ ]:
sample(connection, 'outcomes')

## Combine with Redcap
Now that we have some mimic iii fields translated to ecmocard fields, lets combine them with redcap data to augment our dataset.

### Create features and labels
We will use the comorbidities as features and the outcomes field as a label to create an XGBoost model to predict mortality.

#### Get RedCap features

In [ ]:
features = 'malignantneo_mhyn,diabetes_mhyn,diabetiscomp_mhyn,obesity_mhyn,chronhaemo_mhyn,chronicneu_mhyn,'+\
           'dementia_mhyn,chroniccard_mhyn,aidshiv_mhyn,chronicpul_mhyn,asthma_mhyn,modliver_mhyn,renal_mhyn,'+\
           'other_mhyn'
transforms = ',\n  '.join([f"if({m} = '1', 1, 0) AS {m}" for m in features.split(',')])

redcap_features = pd.read_sql(f"""
SELECT
    if(dsterm = '4', 1, 0) AS dsterm,
    {transforms},
    if(smoking_mhyn = '1', 1, 0) AS smoker,
    if(smoking_mhyn = '3', 1, 0) AS former_smoker
FROM redcap.etl_redcap
""", connection)
redcap_features.head(5)

#### Get the mimic iii features

In [ ]:
transforms = ',\n  '.join([f"if({m} = 1, 1, 0) AS {m}" for m in features.split(',')])
mimic_features = pd.read_sql(f'''
SELECT
    if(dsterm = 4, 1, 0) AS dsterm,
    {transforms},
    if(smoking_mhyn = 1, 1, 0) AS smoker,
    if(smoking_mhyn = 3, 1, 0) AS former_smoker
FROM echmocard.comorbidities AS c
JOIN echmocard.outcomes AS o ON (c.subject_id = o.subject_id)
''', connection)

mimic_features.head(10)

#### Combine and convert columns to integers

In [ ]:
combined = pd.concat([mimic_features, redcap_features])
combined['dsterm'] = (combined['dsterm'] == 4).astype(int)
for feature in features.split(','):
    combined[feature] = (combined[feature] == True).astype(int)
combined.head(10)

## Create XGBoost model

#### Split into test, train and validation

In [ ]:
rand_split = np.random.rand(len(combined))
train_data = combined[rand_split < 0.8]
validation_data = combined[(rand_split >= 0.8) & (rand_split < 0.9)]
test_data = combined[rand_split >= 0.9]

#### Write data to S3

In [ ]:
INPUT_DATA_PATH = 'ml/comorbidities_xgboost/data'

# function that write files directly to s3
@contextlib.contextmanager
def s3_file(file):
    buffer = StringIO()
    yield buffer
    print(f"writing s3://{S3_BUCKET}/{INPUT_DATA_PATH}/{file}")
    s3 = boto3.resource('s3')
    s3.Object(S3_BUCKET, INPUT_DATA_PATH+'/'+file).put(Body=buffer.getvalue())
    
with s3_file('data.csv') as file: 
    train_data.to_csv(file, index=False, header=True)
    
with s3_file('train.csv') as file: 
    train_data.to_csv(file, index=False, header=False)    

with s3_file('val.csv') as file: 
    validation_data.to_csv(file, index=False, header=False)

with s3_file('test.csv') as file: 
    test_data.to_csv(file, index=False, header=False)



### Fit the model
#### Get training instance
Now we will get the container for XGBoost in our region, as well as a sagemaker session and execution role for training

In [ ]:
container = get_image_uri(boto3.Session().region_name, 'xgboost', '0.90-1')
session = sagemaker.Session()
role = get_execution_role()

#### Create an estimator

In [ ]:
xgb = sagemaker.estimator.Estimator(
    container,
    role, 
    train_instance_count= 1, 
    train_instance_type= 'ml.m4.xlarge',
    output_path= f's3://{S3_BUCKET}/ml/comorbidities_xgboost/model',
    sagemaker_session=session
)

#### Set the known hyperparameters

In [ ]:
xgb.set_hyperparameters(
    subsample=0.8,
    silent=0,
    objective='binary:logistic',
    num_round=100,
    eval_metric='auc'
)

#### Hyperparameter tuning
Now we can create an HPO job to find the best hyperparmarameters by training multiple models.

In [ ]:
hyperparameter_ranges = {'eta': ContinuousParameter(0, 1),
                        'min_child_weight': ContinuousParameter(1, 10),
                        'alpha': ContinuousParameter(0, 2),
                        'max_depth': IntegerParameter(1, 10)}

hpo_tuner = HyperparameterTuner(
    xgb,
    'validation:auc',
    hyperparameter_ranges,
    max_jobs=20,
    max_parallel_jobs=1,
    strategy='Bayesian'
)

s3_input_train = sagemaker.s3_input(s3_data=f's3://{S3_BUCKET}/{INPUT_DATA_PATH}/train.csv', content_type='csv')
s3_input_val = sagemaker.s3_input(s3_data=f's3://{S3_BUCKET}/{INPUT_DATA_PATH}/val.csv', content_type='csv')

In [ ]:
hpo_tuner.fit({'train': s3_input_train, 'validation': s3_input_val}, include_cls_metadata=False)

In [ ]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=hpo_tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

In [ ]:
! git 